In [1]:
!pip install transformers datasets accelerate bitsandbytes


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install protobuf sentencepiece


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [6]:
import random
import json

# Define some sample features and values to generate synthetic data
features = [
    "AMT_INCOME_TOTAL_NEGATIVE", "AMT_INCOME_TOTAL_POSITIVE", "AMT_CREDIT_NEGATIVE", "AMT_CREDIT_POSITIVE", 
    "AMT_ANNUITY_NEGATIVE", "AMT_ANNUITY_POSITIVE", "AMT_GOODS_PRICE_NEGATIVE", "AMT_GOODS_PRICE_POSITIVE"
]

# Define explanation templates
explanation_templates = [
    "This feature indicates {explanation}.",
    "The value signifies {explanation}.",
    "The feature reflects {explanation}.",
    "We can conclude that {explanation}.",
]

# Define complex explanations for ALL features
complex_explanations = {
    "AMT_INCOME_TOTAL_NEGATIVE": [
        "that the client has a high annual income. This means that the client has better financial stability and cash flow, which decreases default risk",
        "that there is a lower default risk, as the client is more stable financially and has a better cash flow due to a high annual income"
    ],
    "AMT_INCOME_TOTAL_POSITIVE": [
        "that the client has a low annual income. This means that the client does not have a good financial stability and cash flow, which increases default risk",
        "that there is a higher default risk, as the client is less stable financially and does not have a good cash flow due to a low annual income"
    ],
    "AMT_CREDIT_NEGATIVE": [
        "that the loan amount aligns with the client's annual income, indicating responsible borrowing. This is a positive indicator of the client's financial health, as it suggests they are not overextending themselves financially",
        "the client practices prudent borrowing due to the alignment of the loan amount with the annual income of the client. This is a favorable sign of the client's financial stability due to good financial management"
    ],
    "AMT_CREDIT_POSITIVE": [
        "that the loan amount does not align with the client's annual income, indicating irresponsible borrowing. This is a negative indicator of the client's financial health, as it suggests they might be overextending themselves financially",
        "the client does not practice prudent borrowing due to the misalignment of the loan amount with the client's annual income. This is not a favorable sign of the client's financial stability due to bad financial management"
    ],
    "AMT_ANNUITY_NEGATIVE": [
        "that the client is able to fulfill a reasonable yearly repayment for the loan, which decreases the risk of defaulting",
        "that the client has the ability to meet annual loan repayments, reducing the likelihood of the client defaulting on the loan"
    ],
    "AMT_ANNUITY_POSITIVE": [
        "that the client is not able to fulfill a reasonable yearly repayment for the loan, which increases the risk of defaulting",
        "that the client might have difficulties meeting annual loan repayments, increasing the likelihood of the client defaulting on the loan"
    ],
    "AMT_GOODS_PRICE_NEGATIVE": [
        "that the good's cost aligns with the loan amount, indicating responsible borrowing. This is a positive indicator of the client's financial health, as it suggests they are not overextending themselves financially",
        "the client practices prudent borrowing due to the alignment of the goods' price with the loan amount. This is a favorable sign of the client's financial stability due to good financial management"
    ],
    "AMT_GOODS_PRICE_POSITIVE": [
        "that the good's cost does not align with the loan amount, indicating irresponsible borrowing. This is a negative indicator of the client's financial health, as it suggests they might be overextending themselves financially",
        "the client does not practice prudent borrowing due to the misalignment of the goods' price with the loan amount. This is not a favorable sign of the client's financial stability due to bad financial management"
    ]
}

# Function to generate synthetic data
def generate_synthetic_data(num_samples=100):
    synthetic_data = []
    for _ in range(num_samples):
        feature = random.choice(features)
        value = f"{feature}"  # Randomize value
        
        # Always use specific complex explanations
        explanation = random.choice(complex_explanations[feature])
        
        # Apply a random explanation template
        explanation_template = random.choice(explanation_templates)
        output = explanation_template.format(value=value, explanation=explanation)

        # Add the data point
        data_point = {
            "instruction": "Explain the feature.",
            "input": value,
            "output": output
        }
        synthetic_data.append(data_point)
    
    return synthetic_data

# Generate synthetic dataset
num_samples = 100000  # Increase the number of samples for better generalization
synthetic_dataset = generate_synthetic_data(num_samples)

# Split dataset into training and validation (80% training, 20% validation)
train_size = int(0.8 * len(synthetic_dataset))  # 80% training data
train_dataset = synthetic_dataset[:train_size]
val_dataset = synthetic_dataset[train_size:]

# Save the datasets to JSON files
with open("train_dataset.json", "w") as f:
    json.dump(train_dataset, f)

with open("val_dataset.json", "w") as f:
    json.dump(val_dataset, f)

print("Synthetic dataset generated and split into train and validation datasets.")


Synthetic dataset generated and split into train and validation datasets.


In [7]:
print(val_dataset[0:2])

[{'instruction': 'Explain the feature.', 'input': 'AMT_CREDIT_NEGATIVE', 'output': "The feature reflects that the loan amount aligns with the client's annual income, indicating responsible borrowing. This is a positive indicator of the client's financial health, as it suggests they are not overextending themselves financially."}, {'instruction': 'Explain the feature.', 'input': 'AMT_GOODS_PRICE_POSITIVE', 'output': "The feature reflects that the good's cost does not align with the loan amount, indicating irresponsible borrowing. This is a negative indicator of the client's financial health, as it suggests they might be overextending themselves financially."}]


In [9]:
print(synthetic_dataset[1])

{'instruction': 'Explain the feature.', 'input': 'AMT_ANNUITY_NEGATIVE', 'output': 'We can conclude that that the client is able to fulfill a reasonable yearly repayment for the loan, which decreases the risk of defaulting.'}


In [10]:
# !nvidia-smi

Mon Jan 13 15:09:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       On  |   00000000:E7:00.0 Off |                    0 |
| N/A   38C    P0             27W /   70W |     201MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [12]:
import json
from datasets import load_dataset
from transformers import AutoTokenizer

# Save the synthetic dataset and validation dataset to JSON files
with open("synthetic_dataset.json", "w") as f:
    json.dump(synthetic_dataset, f)

# Assuming you have a validation dataset named synthetic_dataset_validation.json
# Save the validation dataset
with open("val_dataset.json", "w") as f:
    json.dump(val_dataset, f)

# Load the dataset
raw_dataset = load_dataset("json", data_files={"train": "synthetic_dataset.json", "validation": "val_dataset.json"})

# Tokenizer initialization
model_name = "t5-small"  # You can replace this with a larger model if necessary
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)  # Use fast tokenizer for better performance

# Preprocessing function
def preprocess_function(examples):
    # Format the input as "Instruction: {instruction}\nInput: {input}"
    inputs = [f"Instruction: {instr}\nInput: {inp}" for instr, inp in zip(examples["instruction"], examples["input"])]
    outputs = examples["output"]
    
    # Tokenize inputs and outputs
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length", return_tensors="pt")
    labels = tokenizer(outputs, max_length=512, truncation=True, padding="max_length", return_tensors="pt")
    
    # Replace `labels` with the tokenized output in the correct format
    model_inputs["labels"] = labels["input_ids"]

    # Ensure the attention masks are included (which will be added automatically by `return_tensors="pt"`)
    model_inputs["attention_mask"] = model_inputs["attention_mask"].squeeze().tolist()

    # Make sure the labels are not padded (optional, but can help)
    model_inputs["labels"] = model_inputs["labels"].squeeze().tolist()

    return model_inputs

# Tokenize the entire dataset (train and validation)
tokenized_dataset = raw_dataset.map(preprocess_function, batched=True, remove_columns=["instruction", "input", "output"])

# Check the first tokenized sample from the training set
print(tokenized_dataset["train"][0])

# Check the first tokenized sample from the validation set
print(tokenized_dataset["validation"][0])


/opt/app-root/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100000 examples [00:00, 169903.98 examples/s]
Generating validation split: 20000 examples [00:00, 177927.93 examples/s]
Map: 100%|██████████| 20000/20000 [00:10<00:00, 1974.86 examples/s]

{'input_ids': [21035, 10, 25488, 8, 1451, 5, 86, 2562, 10, 71, 7323, 834, 3162, 6657, 427, 834, 5647, 16359, 834, 16034, 3177, 8087, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [13]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Mon Jan 13 15:10:42 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       On  |   00000000:E7:00.0 Off |                    0 |
| N/A   38C    P0             27W /   70W |     201MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [14]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://download.pytorch.org/whl/cu124


In [15]:
import torch
import torchvision
import torchaudio

print(f"PyTorch Version: {torch.__version__}")
print(f"torchvision Version: {torchvision.__version__}")
print(f"torchaudio Version: {torchaudio.__version__}")
print(f"CUDA Version: {torch.version.cuda}")
print(f"GPU Available: {torch.cuda.is_available()}")


PyTorch Version: 2.5.1+cu124
torchvision Version: 0.20.1+cu124
torchaudio Version: 2.5.1+cu124
CUDA Version: 12.4
GPU Available: True


In [17]:
from transformers import AutoModelForSeq2SeqLM

# Load the model with 8-bit quantization
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    load_in_8bit=True,  # Load the model in 8-bit
    device_map="balanced",  # Automatically allocate model layers to GPUs
    torch_dtype=torch.float16,  # Load model in float16 precision
)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [18]:
!nvidia-smi

Mon Jan 13 15:10:49 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       On  |   00000000:E7:00.0 Off |                    0 |
| N/A   38C    P0             28W /   70W |     333MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
!pip install peft


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [20]:
from peft import LoraConfig, get_peft_model

# Configure LoRA adapters
peft_config = LoraConfig(
    task_type="SEQ_2_SEQ_LM",  # Task type: Causal Language Modeling
    inference_mode=False,   # Enable training
    r=8,                    # Rank of the adaptation matrices
    lora_alpha=32,          # Scaling factor for LoRA
    lora_dropout=0.1,       # Dropout rate for LoRA
)

# Attach LoRA adapters to the quantized model
model = get_peft_model(model, peft_config)

# Verify trainable parameters
model.print_trainable_parameters()


trainable params: 294,912 || all params: 60,801,536 || trainable%: 0.4850


In [22]:
!nvidia-smi

Mon Jan 13 15:10:53 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       On  |   00000000:E7:00.0 Off |                    0 |
| N/A   38C    P0             27W /   70W |     333MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
# from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

# training_args = TrainingArguments(
#     output_dir="./flanT5-fine-tuned",  # Directory to save the model
#     evaluation_strategy="no",          # No evaluation (since we don't have a validation set)
#     save_steps=500,                    # Save the model every 200 steps
#     logging_steps=150,                  # Log training progress every 50 steps
#     per_device_train_batch_size=5,     # Batch size per device
#     gradient_accumulation_steps=4,     # Accumulate gradients for larger batch sizes
#     num_train_epochs=5,               # Number of epochs
#     learning_rate=5e-5,                # Learning rate
#     fp16=True,                         # Enable mixed precision
#     save_total_limit=2,                # Keep only the last 2 model checkpoints
#     report_to="none",                  # Disable reporting to third-party platforms
# )



In [24]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

# Update the training arguments
training_args = TrainingArguments(
    output_dir="./flanT5-fine-tuned",  # Directory to save the model
    evaluation_strategy="epoch",        # Evaluate at the end of each epoch
    save_strategy="epoch",             # Save checkpoint at the end of each epoch
    save_steps=500,                    # Save the model every 500 steps (if you want it in between epochs)
    logging_steps=150,                  # Log training progress every 150 steps
    per_device_train_batch_size=8,     # Batch size per device
    gradient_accumulation_steps=2,     # Accumulate gradients for larger batch sizes
    num_train_epochs=2,                # Number of epochs
    learning_rate=5e-5,                # Learning rate
    fp16=True,                         # Enable mixed precision
    save_total_limit=2,                # Keep only the last 2 model checkpoints
    report_to="none",                  # Disable reporting to third-party platforms
    load_best_model_at_end=True,       # Load the best model when finished training
    metric_for_best_model="eval_loss", # Metric to monitor for the best model
)

# Early stopping callback to stop training early if the validation loss is not improving
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=2)  # Stop after 2 non-improving evals


/opt/app-root/lib64/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [26]:
model.print_trainable_parameters()


trainable params: 294,912 || all params: 60,801,536 || trainable%: 0.4850


In [29]:
from transformers import Trainer

# Trainer initialization
trainer = Trainer(
    model=model,                       # Replace with your model
    args=training_args,
    train_dataset=tokenized_dataset["train"],  # Your training dataset
    eval_dataset=tokenized_dataset["validation"],  # Your validation dataset
    callbacks=[early_stopping_callback],  # Add early stopping callback
)

# Start the training process
trainer.train()



Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.048100,0.007431
2,0.028300,0.005221


TrainOutput(global_step=12500, training_loss=0.27805256794929506, metrics={'train_runtime': 11312.8672, 'train_samples_per_second': 17.679, 'train_steps_per_second': 1.105, 'total_flos': 2.72495542272e+16, 'train_loss': 0.27805256794929506, 'epoch': 2.0})

In [31]:
trainer.save_model("./flanT5-fine-tuned")
tokenizer.save_pretrained("./flanT5-fine-tuned")


('./flanT5-fine-tuned/tokenizer_config.json',
 './flanT5-fine-tuned/special_tokens_map.json',
 './flanT5-fine-tuned/spiece.model',
 './flanT5-fine-tuned/added_tokens.json',
 './flanT5-fine-tuned/tokenizer.json')

In [318]:
import os

# Set the desired GPU device (e.g., GPU 1)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [319]:
import os
print(os.environ.get("CUDA_VISIBLE_DEVICES"))


0


In [320]:
import torch

print(f"Is CUDA available: {torch.cuda.is_available()}")
print(f"Current Device: {torch.cuda.current_device()}")
print(f"Device Name: {torch.cuda.get_device_name(torch.cuda.current_device())}")


Is CUDA available: True
Current Device: 0
Device Name: Tesla T4


## Model Testing

In [321]:
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

# Load model with offloading
fine_tuned_model = AutoModelForSeq2SeqLM.from_pretrained("./flanT5-fine-tuned")

fine_tuned_tokenizer = AutoTokenizer.from_pretrained("./flanT5-fine-tuned")


generator = pipeline("text2text-generation", model=fine_tuned_model, tokenizer=fine_tuned_tokenizer)

# Test with a single input
test_input = "Instruction: Explain the feature.\nInput: AMT_CREDIT_POSITIVE"

output = generator(test_input, max_length=100, num_return_sequences=1)
print(output)


Device set to use cuda:0


[{'generated_text': "The value signifies that the loan amount does not align with the client's annual income, indicating irresponsible borrowing. This is a negative indicator of the client's financial health, as it suggests they might be overextending themselves financially."}]


## Saving Model as non-torchscript

In [301]:
fine_tuned_model.to('cpu')  # Move model to CPU
torch.save(fine_tuned_model.state_dict(), "model_repository/t5-small/1/flanT5_fine_tuned_cpu.pth")
print("file saved")

file saved


## Trace Model v1 (saved to cpu)

In [322]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load your fine-tuned model and tokenizer
model_path = "./flanT5-fine-tuned"
fine_tuned_model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
fine_tuned_model.eval()  # Set the model to evaluation mode
fine_tuned_tokenizer = AutoTokenizer.from_pretrained(model_path)

# Dummy inputs for TorchScript tracing
dummy_input_ids = torch.ones((1, 128), dtype=torch.long)  # Example input IDs
dummy_attention_mask = torch.ones((1, 128), dtype=torch.long)  # Example attention mask
dummy_decoder_input_ids = torch.ones((1, 1), dtype=torch.long)  # Start of sequence token

# Wrapper for the model to ensure proper handling of inputs and outputs
class T5Wrapper(torch.nn.Module):
    def __init__(self, model):
        super(T5Wrapper, self).__init__()
        self.model = model

    def forward(self, input_ids, attention_mask, decoder_input_ids):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids)
        return outputs.logits

# Wrap the model
wrapped_model = T5Wrapper(fine_tuned_model)

# Trace the model with TorchScript
traced_model = torch.jit.trace(
    wrapped_model, 
    (dummy_input_ids, dummy_attention_mask, dummy_decoder_input_ids),
    strict=False  # Disable strict tracing to allow dynamic behavior
)

# Save the TorchScript model
torchscript_model_path = "model_repository/t5-small/1/T5_torchscript.pt"
traced_model.save(torchscript_model_path)
print(f"TorchScript model saved to {torchscript_model_path}")

# Test the traced model
def test_traced_model(model_path, tokenizer, device):
    # Load the TorchScript model
    loaded_model = torch.jit.load(model_path)
    loaded_model.eval()
    
    # Test inputs
    test_input = "Instruction: Explain the feature.\nInput: AMT_CREDIT_POSITIVE"
    tokenized_input = tokenizer(
        test_input,
        return_tensors="pt",
        max_length=128,
        padding="max_length",
        truncation=True
    )
    
    # Move inputs to GPU
    input_ids = tokenized_input["input_ids"]
    attention_mask = tokenized_input["attention_mask"]
    decoder_input_ids = torch.tensor([[tokenizer.pad_token_id]])
    
    # Iterative decoding loop
    generated_ids = []
    max_length = 80
    
    for _ in range(max_length):
        with torch.no_grad():
            logits = loaded_model(input_ids, attention_mask, decoder_input_ids)
            next_token_id = torch.argmax(logits[:, -1, :], dim=-1)
            generated_ids.append(next_token_id.item())
            
            if next_token_id.item() == tokenizer.eos_token_id:
                break
                
            decoder_input_ids = torch.cat([
                decoder_input_ids,
                next_token_id.unsqueeze(0)
            ], dim=-1)
    
    decoded_output = tokenizer.decode(generated_ids, skip_special_tokens=True)
    return decoded_output

# Test the model
print("Testing traced model...")
output = test_traced_model(torchscript_model_path, fine_tuned_tokenizer, device)
print(f"Predicted Output: {output}")

TorchScript model saved to model_repository/t5-small/1/T5_torchscript.pt
Testing traced model...
Predicted Output: The value signifies that the loan amount does not align with the client's annual income, indicating irresponsible borrowing. This is a negative indicator of the client's financial health, as it suggests they might be overextending themselves financially.


## Trace Model v2

In [283]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load your fine-tuned model and tokenizer
model_path = "./flanT5-fine-tuned"
fine_tuned_model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
fine_tuned_model.to("cuda")  # Move the model to GPU
fine_tuned_model.eval()  # Set the model to evaluation mode
fine_tuned_tokenizer = AutoTokenizer.from_pretrained(model_path)

# Dummy inputs for TorchScript tracing
dummy_input_ids = torch.ones((1, 128), dtype=torch.long).to("cuda")  # Move to GPU
dummy_attention_mask = torch.ones((1, 128), dtype=torch.long).to("cuda")  # Move to GPU
dummy_decoder_input_ids = torch.ones((1, 1), dtype=torch.long).to("cuda")  # Move to GPU

# Wrapper for the model to ensure proper handling of inputs and outputs
class T5Wrapper(torch.nn.Module):
    def __init__(self, model):
        super(T5Wrapper, self).__init__()
        self.model = model

    def forward(self, input_ids, attention_mask, decoder_input_ids):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids)
        return outputs.logits

# Wrap the model
wrapped_model = T5Wrapper(fine_tuned_model)

# Trace the model with TorchScript
traced_model = torch.jit.trace(
    wrapped_model, 
    (dummy_input_ids, dummy_attention_mask, dummy_decoder_input_ids),
    strict=False  # Disable strict tracing to allow dynamic behavior
)

# Save the TorchScript model
torchscript_model_path = "model_repository/t5-small/1/T5_torchscript.pt"
traced_model.save(torchscript_model_path)
print(f"TorchScript model saved to {torchscript_model_path}")


TorchScript model saved to model_repository/t5-small/1/T5_torchscript.pt


## Trace Model v3 (to GPU)

In [327]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load your fine-tuned model and tokenizer
model_path = "./flanT5-fine-tuned"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load model and move to GPU
fine_tuned_model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
fine_tuned_model.eval()
fine_tuned_tokenizer = AutoTokenizer.from_pretrained(model_path)

# Create dummy inputs on GPU
dummy_input_ids = torch.ones((1, 128), dtype=torch.long).to(device)
dummy_attention_mask = torch.ones((1, 128), dtype=torch.long).to(device)
dummy_decoder_input_ids = torch.ones((1, 1), dtype=torch.long).to(device)

class T5Wrapper(torch.nn.Module):
    def __init__(self, model):
        super(T5Wrapper, self).__init__()
        self.model = model
    
    def forward(self, input_ids, attention_mask, decoder_input_ids):
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids
        )
        return outputs.logits

# Wrap the model and ensure it's on GPU
wrapped_model = T5Wrapper(fine_tuned_model).to(device)

# Trace the model with TorchScript
traced_model = torch.jit.trace(
    wrapped_model,
    (dummy_input_ids, dummy_attention_mask, dummy_decoder_input_ids),
    strict=False
)

# Save the TorchScript model
torchscript_model_path = "model_repository/t5-small/1/T5_torchscript.pt"
traced_model.save(torchscript_model_path)
print(f"TorchScript model saved to {torchscript_model_path}")

# Test the traced model
def test_traced_model(model_path, tokenizer, device):
    # Load the TorchScript model
    loaded_model = torch.jit.load(model_path).to(device)
    loaded_model.eval()
    
    # Test inputs
    test_input = "Instruction: Explain the feature.\nInput: AMT_CREDIT_POSITIVE"
    tokenized_input = tokenizer(
        test_input,
        return_tensors="pt",
        max_length=128,
        padding="max_length",
        truncation=True
    )
    
    # Move inputs to GPU
    input_ids = tokenized_input["input_ids"].to(device)
    attention_mask = tokenized_input["attention_mask"].to(device)
    decoder_input_ids = torch.tensor([[tokenizer.pad_token_id]], device=device)
    
    # Iterative decoding loop
    generated_ids = []
    max_length = 80
    
    for _ in range(max_length):
        with torch.no_grad():
            logits = loaded_model(input_ids, attention_mask, decoder_input_ids)
            next_token_id = torch.argmax(logits[:, -1, :], dim=-1)
            generated_ids.append(next_token_id.item())
            
            if next_token_id.item() == tokenizer.eos_token_id:
                break
                
            decoder_input_ids = torch.cat([
                decoder_input_ids,
                next_token_id.unsqueeze(0)
            ], dim=-1)
    
    decoded_output = tokenizer.decode(generated_ids, skip_special_tokens=True)
    return decoded_output

# Test the model
print("Testing traced model...")
output = test_traced_model(torchscript_model_path, fine_tuned_tokenizer, device)
print(f"Predicted Output: {output}")

TorchScript model saved to model_repository/t5-small/1/T5_torchscript.pt
Testing traced model...
Predicted Output: The value signifies that the loan amount does not align with the client's annual income, indicating irresponsible borrowing. This is a negative indicator of the client's financial health, as it suggests they might be overextending themselves financially.


### Model Serving and Testing

In [328]:
import os
import boto3
import botocore

# Retrieve AWS credentials and S3 connection details from environment variables
aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.environ.get('AWS_S3_ENDPOINT')
region_name = os.environ.get('AWS_DEFAULT_REGION')
bucket_name = os.environ.get('AWS_S3_BUCKET')

# Check if all required environment variables are set
if not all([aws_access_key_id, aws_secret_access_key, endpoint_url, region_name, bucket_name]):
    raise ValueError("One or more data connection variables are empty. "
                     "Please check your data connection to an S3 bucket.")

# Create a session and S3 resource
session = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                aws_secret_access_key=aws_secret_access_key)

s3_resource = session.resource(
    's3',
    config=botocore.client.Config(signature_version='s3v4'),
    endpoint_url=endpoint_url,
    region_name=region_name)

bucket = s3_resource.Bucket(bucket_name)


def upload_file_to_s3(local_file_path, s3_key):
    """
    Uploads a single file to S3.
    
    Parameters:
    - local_file_path: Path to the local file to upload.
    - s3_key: Path within the S3 bucket to store the file.
    """
    if not os.path.isfile(local_file_path):
        raise ValueError(f"The file '{local_file_path}' does not exist.")
    
    print(f"Uploading {local_file_path} to s3://{bucket_name}/{s3_key}")
    bucket.upload_file(local_file_path, s3_key)
    print(f"File uploaded successfully to s3://{bucket_name}/{s3_key}")
    return 1  # Return 1 since only one file is uploaded


def list_objects(prefix):
    """
    Lists all objects in the S3 bucket with a given prefix.
    
    Parameters:
    - prefix: The prefix to filter objects in the S3 bucket.
    """
    filter = bucket.objects.filter(Prefix=prefix)
    for obj in filter.all():
        print(obj.key)


# Example usage of uploading specific files
# List of files to upload
files_to_upload = [
    "model_repository/t5-small/1/T5_torchscript.pt",  # Specify each file path here
    # "model_repository/t5-small/config.pbtxt",
    # "model_repository/t5-small/1/flanT5_fine_tuned_cpu.pth"
]

# Check if the files exist locally and upload them individually
for file_path in files_to_upload:
    # Extract the relative path within the bucket
    s3_key = "t5-pytorch/1/T5_torchscript.pt"  # Adjust if you want a different path in S3
    upload_file_to_s3(file_path, s3_key)

# List the uploaded files in S3
print("\nUploaded files in S3 under 'models':")
list_objects("models")

Uploading model_repository/t5-small/1/T5_torchscript.pt to s3://test/t5-pytorch/1/T5_torchscript.pt
File uploaded successfully to s3://test/t5-pytorch/1/T5_torchscript.pt

Uploaded files in S3 under 'models':
models/1/xgb_classifier.onnx
models/vicuna/README.md
models/vicuna/config.json
models/vicuna/generation_config.json
models/vicuna/pytorch_model-00001-of-00002.bin
models/vicuna/pytorch_model-00002-of-00002.bin
models/vicuna/pytorch_model.bin.index.json
models/vicuna/special_tokens_map.json
models/vicuna/tokenizer.model
models/vicuna/tokenizer_config.json


In [338]:
import requests
import torch
from transformers import AutoTokenizer

# Initialize tokenizer
model_path = "./flanT5-fine-tuned"
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Define inference URL
infer_url = "http://modelmesh-serving.llm.svc.cluster.local:8008/v2/models/t5-pytorch/infer"

# Test input text
input_text = "Instruction: Explain the feature.\nInput: AMT_INCOME_TOTAL_POSITIVE"

# Tokenize the input
tokenized_input = tokenizer(
    input_text,
    return_tensors="pt",
    max_length=128,
    padding="max_length",
    truncation=True
)

# Extract tokenized tensors
input_ids = tokenized_input["input_ids"]
attention_mask = tokenized_input["attention_mask"]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)

# Initialize decoder input IDs with <pad> token
decoder_input_ids = torch.tensor([[tokenizer.pad_token_id]], device=device)

# Iterative decoding
max_length = 80  # Maximum generation length
generated_ids = []

for _ in range(max_length):
    # Prepare payload for the current step
    payload = {
        "inputs": [
            {
                "name": "input_ids",
                "shape": list(input_ids.shape),
                "datatype": "INT64",
                "data": input_ids.flatten().tolist()
            },
            {
                "name": "attention_mask",
                "shape": list(attention_mask.shape),
                "datatype": "INT64",
                "data": attention_mask.flatten().tolist()
            },
            {
                "name": "decoder_input_ids",
                "shape": list(decoder_input_ids.shape),
                "datatype": "INT64",
                "data": decoder_input_ids.flatten().tolist()
            }
        ]
    }

    # Send POST request
    headers = {"Content-Type": "application/json"}
    response = requests.post(infer_url, json=payload, headers=headers, verify=False)

    if response.status_code == 200:
        response_data = response.json()
        logits = response_data["outputs"][0]["data"]
        logits_tensor = torch.tensor(logits, device=device).view(1, -1, 32128)

        # Get the next token
        next_token_id = torch.argmax(logits_tensor[:, -1, :], dim=-1)
        generated_ids.append(next_token_id.item())

        # Break if EOS token is generated
        if next_token_id.item() == tokenizer.eos_token_id:
            break

        # Update decoder_input_ids for the next iteration
        decoder_input_ids = torch.cat([decoder_input_ids, next_token_id.unsqueeze(0)], dim=-1)

    else:
        print(f"Error {response.status_code}: {response.text}")
        break

# Decode the generated IDs to text
generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
print("Generated Output:", generated_text)


Generated Output: The feature reflects that there is a higher default risk, as the client is less stable financially and does not have a good cash flow due to a low annual income.


## Model Testing v2 (Health Check)

In [329]:
import requests

infer_url = "http://modelmesh-serving.llm.svc.cluster.local:8008/v2/models/t5-pytorch/infer"


payload = {
    "inputs": [
        {"name": "input_ids", "shape": [1, 128], "datatype": "INT64", "data": input_ids.flatten().tolist()},
        {"name": "attention_mask", "shape": [1, 128], "datatype": "INT64", "data": attention_mask.flatten().tolist()},
        {"name": "decoder_input_ids", "shape": [1, 1], "datatype": "INT64", "data": decoder_input_ids.flatten().tolist()}
    ]
}

response = requests.post(infer_url, json=payload)
print(response.json())


{'model_name': 't5-pytorch__isvc-fb25ed5859', 'model_version': '1', 'outputs': [{'name': 'logits', 'datatype': 'FP32', 'shape': [1, 1, 32128], 'data': [-15.494759, -12.542038, -16.291527, -3.170318, -21.694927, -11.366928, -12.663254, -14.082267, -6.5090137, -16.538254, -12.606465, -11.527391, -14.569662, -14.425108, -21.170452, -18.164848, -13.050201, -19.128632, -13.34326, -13.447127, -13.069403, -13.6678505, -15.8760395, -19.644947, -13.951824, -16.62828, -18.069859, -3.7137904, -15.4413395, -17.850004, -14.410351, -15.649021, -20.106474, -17.219717, -12.849901, -18.359764, -17.545961, 2.886934, -14.308861, -17.212011, -17.636395, -6.5761094, -14.601379, -17.166756, -15.673424, -16.441212, -15.792196, -15.674684, -9.170116, -19.704153, -9.70214, -20.160786, -19.089394, -18.978697, -16.96722, -14.9667635, -16.126347, -15.612015, -14.827377, -14.120045, -20.010382, -13.990071, -10.408994, -21.436522, -13.675734, -15.986326, -17.296427, -12.243713, -13.374195, -12.2648945, -18.257683, 